In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
from time import sleep
import re, math

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [8]:
town = 'woodbury'

list_link = []  
list_price = []  
price_tags = []  
list_date = [] 
list_address = [] 
list_beds = [] 
list_baths = [] 
list_sqft = [] 
list_lotsize = [] 
list_garage = [] 
list_neighbor = [] 
list_style = []
list_year = []
list_month = []
list_lotarea = []

In [9]:

url = 'http://www.realtor.com/soldhomeprices/' + town + '_NY/beds-3/baths-2/price-na-900000/sby-1?pgsz=50' # page1
#url = 'http://www.realtor.com/soldhomeprices/' + town + '_NY/beds-3/baths-2/price-na-900000/sby-1/pg-2?pgsz=50' # page2
#url = 'http://www.realtor.com/soldhomeprices/' + town + '_NY/beds-3/baths-2/price-na-900000/sby-1/pg-3?pgsz=50' # page3

In [10]:
r = requests.get(url, headers=headers )
data = r.text
soup = BeautifulSoup(data)

In [12]:

# links 
links = soup.find_all("div", { "class" : "srp-item-address ellipsis" })
for link in links:
    href = 'http://www.realtor.com/' + link.a['href']
    list_link.append(href)

# price, sold date 
prices = soup.find_all("div", { "class" : "srp-item-price" })

for price in prices:
    sold_info = price.find_all(text = True)
    price_tag = sold_info[0].strip()
    try: sold_price = int(sold_info[0].strip()[1:-4])*1000
    except: sold_price = 0
    sold_date = sold_info[3]
    sold_year = sold_date.split(' ')[-2]
    sold_month = sold_date.split(' ')[1]
    
    price_tags.append(price_tag)
    list_price.append(sold_price)
    list_date.append(sold_date)
    list_year.append(sold_year)
    list_month.append(sold_month)

# address
addresses = soup.find_all("div", {"class" : "srp-item-address ellipsis"})

for address in addresses:
    addr = address.find_all('span')
    addr_full = []
    for add in addr:
        addr_full.append(add.get_text())
    addr_name = ' '.join(addr_full)
      
    list_address.append(addr_name)

# neighbor

contents = soup.find_all("div", {"class" : "srp-item-details"})
for content in contents:
    # neighbor
    neighbors = content.find_all('div', {'class': 'srp-item-neighborhood ellipsis link-tertiary hidden-xxs hidden-xs'})
    for neighbor in neighbors:
        neigh = neighbor.find_all( text=True)[1]
   
    # style
    styles = content.find_all("span", {"class": "srp-property-type"})
    for style in styles:
        style_type = style.find_all(text=True)[0]
        
    # bed
    proptys = content.find_all("ul", {"class": "property-meta list-horizontal list-style-disc list-spaced"})
    for propty in proptys:        
        propt = propty.find_all(text=True)
        bed = propt[1]       
        bath = float(propt[4].replace('+', '.5'))
        sqft = 0
        lot_size = 0
        try:
            if propt[8] == ' sq ft':
                sqft = int(propt[7].replace(',' , ''))
                try: 
                    if propt[11] == ' acres lot':
                        lot_size = float(propt[10])
                        if lot_size > 4.0:
                            lot_size = 0
                    elif propt[11] == ' sq ft lot':
                        try: lot_size = round(float(propt[10].replace(',', ''))/43560.0, 2)
                        except: pass
                    lot_area = propt[10] + propt[11]
                except:
                    pass


            else:
                if propt[8] == ' acres lot':
                    lot_size = float(propt[7])
                    if lot_size > 4.0:
                        lot_size = 0
                elif propt[8] == ' sq ft lot':
                    try: lot_size = round(float(propt[10].replace(',', ''))/43560.0, 2)
                    except: pass

                #lot_area = propt[8] + propt[7]
        except: pass

    
    
    list_neighbor.append(neigh)
    list_style.append(style_type)
    list_beds.append(bed)
    list_baths.append(bath)
    list_sqft.append(sqft)
    list_lotsize.append(lot_size)
    #list_lotarea.append(lot_area)

In [13]:
links = pd.Series(list_link)
dates = pd.Series(list_date)
years = pd.Series(list_year)
months = pd.Series(list_month)
prices = pd.Series(list_price)
pricetags = pd.Series(price_tags)
address = pd.Series(list_address)

neighs = pd.Series(list_neighbor)
types = pd.Series(list_style)
beds = pd.Series(list_beds)
baths = pd.Series(list_baths)
sqfts = pd.Series(list_sqft)
lotsize = pd.Series(list_lotsize)
lotarea = pd.Series(list_lotarea)


In [14]:
tb_house = pd.DataFrame({'address': address,'price' : prices, 'price_tag': pricetags, 'date' : dates, 'link' : links ,
                        'neighbors': neighs, 'types': types, 'bedroom': beds, 'bathroom': baths, 'sq ft': sqfts,
                        'lot size-acres': lotsize, 'month': months, 'year' : years, 'lot area': lotarea})
tb_house[:2]

,address,bathroom,bedroom,date,link,lot area,lot size-acres,month,neighbors,price,price_tag,sq ft,types,year
0,8 Wimbledon Ct Woodbury NY 11797,2.5,3,"May 2, 2017",http://www.realtor.com//realestateandhomes-det...,NaN,0.0,May,Woodbury,410000,"$410,000",1488.0,Condo/Townhome/Row Home/Co-Op,2017
1,15 Village Ln Woodbury NY 11797,2.5,3,"February 10, 2017",http://www.realtor.com//realestateandhomes-det...,NaN,0.0,February,Woodbury,445000,"$445,000",1488.0,Condo/Townhome/Row Home/Co-Op,2017


In [266]:
file_name = 'sold_' + town + '.csv'
tb_house.to_csv(file_name)

In [267]:
tb_house.shape

(34, 14)

In [268]:
print(town)
tb_house.groupby(['year', 'month']).price.agg(['count', 'mean', 'median', np.std])

woodbury


count    mean  median            std
year month                                         
2016 December      6  706500  744500  111718.843527
2017 April         5  681000  585000  169564.147154
     February      8  645375  664000  147035.406328
     January       5  731800  728000   87476.854082
     March         6  557500  522500  132504.716897
     May           4  678750  720000  200098.933863